In [9]:
import pandas as pd
import random
#import torch
#from transformers import AutoTokenizer, AutoModel
#import re
#import string
import numpy as np
#import pydot
#from nltk.corpus import stopwords 
#from nltk.stem import WordNetLemmatizer
#from nltk.stem import LancasterStemmer
#from simpletransformers.classification import MultiLabelClassificationModel
#import logging
#import custom_sentence_tokenizer
import matplotlib.pyplot as plt
#from scipy import stats
#from ast import literal_eval
import pickle
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K
import keras
import gc
from keras.models import model_from_json
from sklearn.metrics import classification_report

In [2]:
#from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Reshape
from keras.layers import GlobalMaxPooling1D,Conv2D, Conv1D, AveragePooling2D, MaxPooling1D, AveragePooling1D, LeakyReLU
from keras.models import Model
from keras.utils.vis_utils import plot_model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import concatenate
#from sklearn.decomposition import PCA
import _pickle as cPickle
import io
import time
import sys
import boto3
from boto3.s3.transfer import TransferConfig
import tensorflow as tf

## 1. Import Embeddings (X) and Labels (Y)

In [3]:
now = time.time()
infile = None
infile = open("embeddings_evaluation",'rb')
embeddings_evaluation = cPickle.load(infile)
infile.close()
del(infile)
print(time.time()-now)
gc.collect()

44.87386226654053


0

In [4]:
now = time.time()
infile = None
infile = open("embeddings_7.1",'rb')
embeddings = cPickle.load(infile)
infile.close()
del(infile)
print(time.time()-now)
gc.collect()

45.81072735786438


0

In [ ]:
embeddings.update(embeddings_evaluation)
pickle.dump( embeddings, open( "/home/ec2-user/SageMaker/embeddings.p", "wb" ) )

In [ ]:
del(embeddings_evaluation)

In [3]:
now = time.time()
infile = None
infile = open("embeddings.p",'rb')
embeddings = cPickle.load(infile)
infile.close()
del(infile)
print(time.time()-now)
gc.collect()

92.77402520179749


0

In [4]:
embeddings_np_complete = np.array(list(embeddings.items()))
del(embeddings)

## 2. Load chapter model and helper functions

In [18]:
#Load chapeter model weights
def call_model(file_name):
    model = None
    json_file = open('/home/ec2-user/SageMaker/Models/model_'+file_name+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    opt = keras.optimizers.Adam(lr=0.001)
    # load weights into new model
    model.load_weights('/home/ec2-user/SageMaker/Models/model_'+file_name+".h5")
    model.trainable = False
    return model

In [11]:
def padding(embeddings):
    max_sentence = 20
    padding = max_sentence - len(embeddings)
    if padding > 0:
        padding_shape = (padding, 256, 768)
        pad = np.zeros(padding*256*768).reshape(padding_shape)
        return np.append(embeddings, pad, axis = 0).astype('float16')
    else:
        return embeddings
    
def split_for_train(HADM_ID_LIST, CHAPTER_PIVOT_DF, embeddings_concat):
    HADM_ID_DF = None
    loaded_model = None
    HADM_ID_DF =  pd.DataFrame(data=HADM_ID_LIST, columns=['HADM_ID'])
    CHAPTER_PIVOT_DF_COPY = HADM_ID_DF.set_index('HADM_ID')\
                    .join(CHAPTER_PIVOT_DF, how='left').copy(deep=False)
    CHAPTER_PIVOT_NP = np.array(CHAPTER_PIVOT_DF_COPY)
    return train_test_split(embeddings_concat, CHAPTER_PIVOT_NP, test_size=0.5)

def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

import tensorflow as tf
def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())
    
    #p1 = tn / (tn + fn + K.epsilon())
    #r1 = tn / (tn + fp + K.epsilon())
    
    f1 = p*r / (p+r+K.epsilon())
    #+ p1*r1 / (p1+r1+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

## 3. Loop on individual therapy area (bag of models)

In [40]:
score_list = []
history_list = []
file_name_LIST = ['001_139.csv','140_239.csv','240_279.csv','280_289.csv','290_319.csv','320_389.csv','390_459.csv'
,'460_519.csv','520_579.csv','580_629.csv','630_679__740_759__760_779.csv', '710-739.csv','780-799.csv','800-999.csv',
                  'E_V.csv', 'model_adverse_effect_2.csv']
1. #file_name_LIST = ['001_139.csv','140_239.csv','240_279.csv','800-999.csv']
#file_name_LIST = ['320_389.csv']
#file_name_LIST = ['630_679__740_759__760_779.csv']

for file_name in file_name_LIST:
    print('\n\n============Started '+file_name+'============')
    CHAPTER_PIVOT_DF = pd.read_csv('Layer-2/'+file_name, sep=',', header = 0)\
                        .astype({'HADM_ID': 'str'}).set_index('HADM_ID')
    HADM_ID_LIST = list(CHAPTER_PIVOT_DF.index)
    embeddings_np = embeddings_np_complete[np.isin(embeddings_np_complete[:,0], HADM_ID_LIST)]
    print(file_name+' orginal shape: '+str(embeddings_np.shape))
    HADM_ID_LIST = embeddings_np[:,0].copy()
    embeddings = embeddings_np[:,1].copy()
    
    #Zero Pad embeddings
    embeddings_padded = [padding(embeddings_ITEM) for embeddings_ITEM in embeddings]
    del(embeddings)
    gc.collect()
    print(file_name+' Padding complete')

    #Reshape embeddings
    #embeddings_concat = np.array([np.concatenate(i) for i in embeddings_padded])
    embeddings_concat = np.array(embeddings_padded).reshape(len(embeddings_padded), 20*256, 768)
    print(file_name+' modified shape: '+str(embeddings_concat.shape))
    del(embeddings_padded)
    gc.collect()
    
    X_train2, X_test2, y_train2, y_test2 = split_for_train(HADM_ID_LIST, CHAPTER_PIVOT_DF, embeddings_concat)
    print('Train and Test Shapre: '+str(X_train2.shape)+', '+str(X_test2.shape)+', '+str(y_train2.shape)+', '+str(y_test2.shape))
    X_val = X_test2[:int(0.25*len(X_test2))]
    y_val = y_test2[:int(0.25*len(y_test2))]
    del(HADM_ID_LIST)
    del(CHAPTER_PIVOT_DF)
    del(embeddings_concat)
    gc.collect()
    model = call_model(file_name)
    opt = keras.optimizers.Adam(lr=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[get_f1, 'accuracy'])
    predict = model.predict(X_test2, verbose=1)
    print(classification_report(y_test2, np.where(predict > 0.5, 1, 0)))
    



============Started 001_139.csv============
001_139.csv orginal shape: (5135, 2)
001_139.csv Padding complete
001_139.csv modified shape: (5135, 5120, 768)
Train and Test Shapre: (2567, 5120, 768), (2568, 5120, 768), (2567, 4), (2568, 4)
2568/2568 [==============================] - 44s 17ms/step
              precision    recall  f1-score   support

           0       0.86      0.97      0.91      1788
           1       0.88      0.52      0.65       631
           2       0.86      0.55      0.67       471
           3       0.98      0.15      0.26       428

   micro avg       0.86      0.72      0.79      3318
   macro avg       0.90      0.55      0.62      3318
weighted avg       0.88      0.72      0.74      3318
 samples avg       0.81      0.75      0.76      3318



============Started 140_239.csv============
140_239.csv orginal shape: (2369, 2)
140_239.csv Padding complete
140_239.csv modified shape: (2369, 5120, 768)
Train and Test Shapre: (1184, 5120, 768), (1185, 5120,

/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


FileNotFoundError: [Errno 2] File Layer-2/model_adverse_effect_2.csv does not exist: 'Layer-2/model_adverse_effect_2.csv'

In [43]:
score_list = []
history_list = []
file_name_LIST = ['adverse_effect_2.csv']
1. #file_name_LIST = ['001_139.csv','140_239.csv','240_279.csv','800-999.csv']
#file_name_LIST = ['320_389.csv']
#file_name_LIST = ['630_679__740_759__760_779.csv']

for file_name in file_name_LIST:
    print('\n\n============Started '+file_name+'============')
    CHAPTER_PIVOT_DF = pd.read_csv('Layer-2/'+file_name, sep=',', header = 0)\
                        .astype({'HADM_ID': 'str'}).set_index('HADM_ID')
    HADM_ID_LIST = list(CHAPTER_PIVOT_DF.index)
    embeddings_np = embeddings_np_complete[np.isin(embeddings_np_complete[:,0], HADM_ID_LIST)]
    print(file_name+' orginal shape: '+str(embeddings_np.shape))
    HADM_ID_LIST = embeddings_np[:,0].copy()
    embeddings = embeddings_np[:,1].copy()
    
    #Zero Pad embeddings
    embeddings_padded = [padding(embeddings_ITEM) for embeddings_ITEM in embeddings]
    del(embeddings)
    gc.collect()
    print(file_name+' Padding complete')

    #Reshape embeddings
    #embeddings_concat = np.array([np.concatenate(i) for i in embeddings_padded])
    embeddings_concat = np.array(embeddings_padded).reshape(len(embeddings_padded), 20*256, 768)
    print(file_name+' modified shape: '+str(embeddings_concat.shape))
    del(embeddings_padded)
    gc.collect()
    
    X_train2, X_test2, y_train2, y_test2 = split_for_train(HADM_ID_LIST, CHAPTER_PIVOT_DF, embeddings_concat)
    print('Train and Test Shapre: '+str(X_train2.shape)+', '+str(X_test2.shape)+', '+str(y_train2.shape)+', '+str(y_test2.shape))
    X_val = X_test2[:int(0.25*len(X_test2))]
    y_val = y_test2[:int(0.25*len(y_test2))]
    del(HADM_ID_LIST)
    del(CHAPTER_PIVOT_DF)
    del(embeddings_concat)
    gc.collect()



============Started adverse_effect_2.csv============
adverse_effect_2.csv orginal shape: (4962, 2)
adverse_effect_2.csv Padding complete
adverse_effect_2.csv modified shape: (4962, 5120, 768)
Train and Test Shapre: (2481, 5120, 768), (2481, 5120, 768), (2481, 5), (2481, 5)


In [44]:
model = call_model(file_name)
opt = keras.optimizers.Adam(lr=0.0001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[get_f1, 'accuracy'])
predict = model.predict(X_test2, verbose=1)
print(classification_report(y_test2, np.where(predict > 0.5, 1, 0)))
    

2481/2481 [==============================] - 39s 16ms/step
              precision    recall  f1-score   support

           0       0.81      0.61      0.69       503
           1       0.67      0.81      0.73       890
           2       0.78      0.72      0.75      1012
           3       0.79      0.69      0.74       812
           4       0.68      0.68      0.68       650

   micro avg       0.73      0.72      0.72      3867
   macro avg       0.74      0.70      0.72      3867
weighted avg       0.74      0.72      0.72      3867
 samples avg       0.72      0.73      0.70      3867

